In [73]:
import numpy as np
import matplotlib.pyplot as plt
from itertools import cycle

from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from scipy import interp
from sklearn.metrics import roc_auc_score

In [74]:
iris = datasets.load_iris()
X = iris.data
y = iris.target
X.shape

(150, 4)

In [75]:
# Binarize the output
y = label_binarize(y, classes=[0, 1, 2])
n_classes = y.shape[1]
print(n_classes)

3


In [76]:
# Add noisy features to make the problem harder
random_state = np.random.RandomState(0)
n_samples, n_features = X.shape
X = np.c_[X, random_state.randn(n_samples, 200 * n_features)]

In [77]:
# shuffle and split training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)
X_train.shape

(75, 804)

In [78]:
# Learn to predict each class against the other
classifier = OneVsRestClassifier(
    svm.SVC(kernel="linear", probability=True, random_state=random_state)
)
y_score = classifier.fit(X_train, y_train).decision_function(X_test)

In [79]:
# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()

In [80]:
y_score[:,1]

array([-0.36482547, -0.63144366, -0.80263073, -0.12395478, -0.27913475,
       -0.34730509, -0.24506467,  0.18264917, -0.24059516, -0.25745136,
       -0.30270719, -0.7030665 , -0.64062258, -0.46354017, -0.2653183 ,
       -0.56706353, -0.04226738, -0.13784913, -0.99673302, -0.91349462,
       -0.06436533, -0.45906496, -0.80086123, -0.70985659, -0.20735021,
       -0.36698232, -0.1159895 , -0.73611276, -0.37676258, -0.71078341,
       -0.41408367, -0.78971942, -0.47617037, -0.46507392, -0.71470221,
       -0.31125083, -0.51388968, -0.0074193 , -0.65773489, -0.60049718,
       -0.21918849, -1.0107504 , -0.60044961, -0.7579771 , -0.81552421,
       -0.84787471, -0.2863326 , -0.43228203, -0.15060119, -0.62804881,
       -0.29572764, -0.34638653, -0.66917752, -0.62483455, -0.11456775,
       -0.34855346, -0.13384038, -0.52487765, -0.23331763, -0.85448531,
       -0.48759987, -0.83392076, -0.81597935, -0.70826295, -0.39894364,
        0.09194626, -0.56572577, -0.46484965, -0.56912947, -0.56

In [84]:
np.amax(y_score[:,1])

0.18264917133489167

In [81]:
for i in range(n_classes):
    print('i is : ', i)
    print(roc_curve(y_test[:, i], y_score[:, i]))
    fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_score[:, i])
    #print(tpr[i].shape)
    roc_auc[i] = auc(fpr[i], tpr[i])

i is :  0
(array([0.        , 0.        , 0.        , 0.01851852, 0.01851852,
       0.03703704, 0.03703704, 0.05555556, 0.05555556, 0.07407407,
       0.07407407, 0.09259259, 0.09259259, 0.12962963, 0.12962963,
       0.14814815, 0.14814815, 0.2037037 , 0.2037037 , 0.27777778,
       0.27777778, 1.        ]), array([0.        , 0.04761905, 0.14285714, 0.14285714, 0.19047619,
       0.19047619, 0.33333333, 0.33333333, 0.38095238, 0.38095238,
       0.61904762, 0.61904762, 0.66666667, 0.66666667, 0.76190476,
       0.76190476, 0.9047619 , 0.9047619 , 0.95238095, 0.95238095,
       1.        , 1.        ]), array([ 1.37991331,  0.37991331,  0.24966175,  0.19599366,  0.18869727,
        0.17642852,  0.12163662,  0.12127427,  0.11801481,  0.06910059,
       -0.01116192, -0.02392675, -0.03259341, -0.04953425, -0.06196433,
       -0.07856729, -0.11107635, -0.1906155 , -0.19299299, -0.23036784,
       -0.25045747, -1.11907501]))
i is :  1
(array([0.        , 0.        , 0.        , 0.02222222